# Where's the Best Place to Sell Books Online?
The e-commerce platform Flipkart hit an all-important milestone in 2018.
After beating out the world's largest online business, Amazon, for holding the top position in India's online retail market, the startup was acquired for by Wal-Mart for $18B USD.

It is commonly believed that startups can beat out larger businesses through differentiation, or being unique.

Will a machine learning classifier show us the differences between books being sold on Amazon vs. Flipkart?

# Setup for Data Analysis and ML Classifications

In [116]:
# Exploratory Data Analysis libraries
import pandas as pd
import numpy as np
# visualizations
import seaborn as sns 
# run Matplotlib graphs inline in notebook
import matplotlib.pyplot as plt  
%matplotlib inline
# Machine Learning libraries 
import sklearn
from sklearn.model_selection import train_test_split  # function used in building ML models

### Instantiating pandas DataFrames

In [117]:
amazon_df = pd.read_csv('Data/amazon.csv')
flipkart_df = pd.read_csv('Data/flipkart.csv')
# View the data for book listings on Amazon
amazon_df.head()

,amazon_title,amazon_author,amazon_rating,amazon_reviews count,amazon_isbn-10,amazon_price
0,Tell Me your Dreams,by Sidney Sheldon,4.4,160.0,8172234902,209
1,The Boy in the Striped Pyjamas (Definitions),by John Boyne,4.6,134.0,1862305277,350
2,Romancing the Balance Sheet: For Anyone Who Ow...,by Anil Lamba,4.5,156.0,9350294311,477
3,Mossad,by Michael Bar-Zohar - Import,4.6,637.0,8184958455,340
4,My Story,by Kamala Das,4.5,42.0,8172238975,178


In [118]:
# do the same for Flipkart books sold
flipkart_df.head()

,flipkart_author,flipkart_isbn10,flipkart_title,flipkart_ratings count,flipkart_price,flipkart_stars
0,Sidney Sheldon,8172234902,TELL ME YOUR DREAMS,902,209,4.5
1,,1862305277,The Boy in the Striped Pyjamas,83,372,4.5
2,Anil Lamba,9350294311,ROMANCING THE BALANCE SHEET,352,477,4.5
3,Bar-Zohar Michael,8184958455,Mossad,560,280,4.5
4,Kamala Das,8172238975,MY STORY,322,178,4.3


## Fixing Data Inconsistencies

Several problems with the data have been identified, as it pertains to this analysis:

1. Columns are named differently across DataFrames
2. The columns are ordered differently across DataFrames
3. Books with the same title have different casing across DataFrames
4. Data type for ratings count is all floats on Amazon data, yet integers for the analogous column on the Flipkart Dataframe.
5. ISBN columns offers seeminly useless information.
6. Different kinds of NaN values exist in the author columns on both DataFrames (i.e. "By NA", "Not Available", or just leaving the cell empty).

### Making Column Names Consistent

In [119]:
def cutoff_prefix(df, prefix_length):
    '''Removes the first prefix_length characters from each column in a DataFrame.'''
    col_names = list(df.columns)
    for i in range(len(col_names)):
        # slicing off the first 7 letters
        label = col_names[i]
        col_names[i] = label[prefix_length:]
    # swap out the names in the df for their shortened versions
    new_df = df
    new_df.columns = col_names
    return new_df

# Store new Dataframes whose column names have been shorted
amzn_df = cutoff_prefix(amazon_df, 7)
fp_df = cutoff_prefix(flipkart_df, 9)

### Combining Data into One

**The Ratings/Reviews Count Column**

As I continue pre-processing data, there are a few things to keep in mind. For one thing, there is no contextual benefit in providing the number of reviews as a floating point number, as opposed to an integer. Therefore, I will convert all the floating point values in the "reviews count" column of the Amazon DataFrame to integer values.

In [120]:
# do the ratings/reviews columns have the same number of values to begin with?
print(len(amzn_df['reviews count']) == len(fp_df['ratings count']))
# let's make sure that both reviews count columns have the same number of non-Nan values
print(len(amzn_df['reviews count'].dropna().values) == len(fp_df['ratings count'].dropna().values))

True
False


In [121]:
# alright, so which df has more values?
print(len(amzn_df['reviews count'].dropna().values))
print(len(fp_df['ratings count'].dropna().values))

1378
1382


Okay, so as a heads-up I will need to fill in some NaN values in the Amazon Dataframe!
Here, I decided to fill in using '0.0'.

In [122]:
amzn_df['reviews count'] = amzn_df['reviews count'].fillna('0.0')

In [123]:
# int conversion
review_counts = list()
for count in amzn_df['reviews count']:
    # removing commas
    # print(count)
    if ',' in count:
        count_ls = [char for char in count if char != ',']
        count = ''.join(count_ls)
    review_counts.append(int(float(count)))                             
    
# replacing the column in the DataFrame
amzn_df['reviews count'] = review_counts

,title,author,rating,reviews count,isbn-10,price
0,Tell Me your Dreams,by Sidney Sheldon,4.4,160,8172234902,209
1,The Boy in the Striped Pyjamas (Definitions),by John Boyne,4.6,134,1862305277,350
2,Romancing the Balance Sheet: For Anyone Who Ow...,by Anil Lamba,4.5,156,9350294311,477
3,Mossad,by Michael Bar-Zohar - Import,4.6,637,8184958455,340
4,My Story,by Kamala Das,4.5,42,8172238975,178
...,...,...,...,...,...,...
1377,Geronimo Stilton and the Kingdom of Fantasy #8...,by Geronimo Stilton,4.6,329,9385887815,274
1378,Harry Potter and the Deathly Hallows (Harry Po...,by J.K. Rowling,4.6,352,1408855712,666
1379,Sita's Ramayana,by Samhita Arni,3.6,5,9380340036,542
1380,The Maze Runner #02 Scorch Trials Movie Tie-in,by James Dashner,3.9,56,9351039684,247


Cool! That was way more complicated than expected. 

For future reference, how about we cross-inspect of all the columns from both DataFrames, so we can:

1. Take note of where Nan values may be an issue,
2. Decide how best to resolve that issues in the context of that column!

To make the for-loop in the next cell functional, I must first resolve the naming conflict between 'reviews count' and 'ratings count', and likewise for 'review

In [128]:
# I assume that with NaN values included, both DataFrames are already the same length.
for col in amzn_df.columns:
    # skipping rating and reviews, since we already did that
    if col != 'reviews count':
        num_valid_rows_am = len(amzn_df[col].dropna().values)
        num_valid_rows_fp = len(fp_df[col].dropna().values)
        print(f"For '{col}', Amazonian data shows {num_valid_rows_am} values, and Flipkart data shows {num_valid_rows_fp} values.")

For 'title', Amazonian data shows 1382 values, and Flipkart data shows 1382 values.
For 'author', Amazonian data shows 1382 values, and Flipkart data shows 1382 values.


KeyError: 'rating'